In [ ]:
# Import libraries
import pandas as pd
import os

# Get current working directory
cwd = '/content/drive/MyDrive/XLLM_study'

# Import AG Dataset
data = pd.read_csv(cwd+'/data/AG_train.csv')
data = pd.DataFrame(data=data)
data.columns = data.columns.str.replace(" ","_")
data.columns = data.columns.str.lower()
data["class_name"] = data["class_index"].map({1:"World", 2:"Sports",
                                              3:"Business", 4:"Sci_Tech"})

In [ ]:
# View data
data

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,World
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports


In [ ]:
# Count observations by class
data.class_name.value_counts()

,count
class_name,
Business,30000
Sci_Tech,30000
Sports,30000
World,30000


In [ ]:
# View titles
for i in range(10):
    print("Title of Article",i)
    print(data.loc[i,"title"])
    print("\n")

Title of Article 0
Wall St. Bears Claw Back Into the Black (Reuters)


Title of Article 1
Carlyle Looks Toward Commercial Aerospace (Reuters)


Title of Article 2
Oil and Economy Cloud Stocks' Outlook (Reuters)


Title of Article 3
Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)


Title of Article 4
Oil prices soar to all-time record, posing new menace to US economy (AFP)


Title of Article 5
Stocks End Up, But Near Year Lows (Reuters)


Title of Article 6
Money Funds Fell in Latest Week (AP)


Title of Article 7
Fed minutes show dissent over inflation (USATODAY.com)


Title of Article 8
Safety Net (Forbes.com)


Title of Article 9
Wall St. Bears Claw Back Into the Black




In [ ]:
# View descriptions
for i in range(10):
    print("Description of Article",i)
    print(data.loc[i,"description"])
    print("\n")

Description of Article 0
Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


Description of Article 1
Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


Description of Article 2
Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.


Description of Article 3
Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.


Description of Article 4
AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.


Description o

In [ ]:
# Clean up text
cols = ["title","description"]
data[cols] = data[cols].applymap(lambda x: x.replace("\\"," "))
data[cols] = data[cols].applymap(lambda x: x.replace("#36;","$"))
data[cols] = data[cols].applymap(lambda x: x.replace("  "," "))
data[cols] = data[cols].applymap(lambda x: x.strip())

<ipython-input-8-1ca99d166875>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cols] = data[cols].applymap(lambda x: x.replace("\\"," "))
<ipython-input-8-1ca99d166875>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cols] = data[cols].applymap(lambda x: x.replace("#36;","$"))
<ipython-input-8-1ca99d166875>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cols] = data[cols].applymap(lambda x: x.replace("  "," "))
<ipython-input-8-1ca99d166875>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cols] = data[cols].applymap(lambda x: x.strip())


In [ ]:
# Write data to CSV
data.to_csv(cwd+'/data/AG_train_prepared.csv', index=False)

# NLP Task #1: Named Entity Recognition

In [1]:
# Load data to CSV
import pandas as pd
cwd = '/content/drive/MyDrive/XLLM_study'
data = pd.read_csv(cwd+'/data/AG_train_prepared.csv')
data

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Pervez...,World
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports


In [29]:
!pip install -U spacy[cuda110,transformers,lookups]==3.0.3
!pip install -U spacy-lookups-data==1.0.0
!pip install cupy-cuda110==8.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 MB 5.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement cupy-cuda110==8.5.0 (from versions: 11.3.0, 11.4.0, 11.5.0, 11.6.0, 12.0.0, 12.1.0, 12.2.0, 12.3.0)
ERROR: No matching distribution found for cupy-cuda110==8.5.0


In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
!pip install spacy[transformers]==3.7.2

  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 54.5 MB/s eta 0:00:00
Using cached smart_open-6.4.0-py3-none-any.whl (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━

In [ ]:
# Import spacy and load language model
import spacy
spacy.require_gpu()
print(spacy.require_gpu())
nlp = spacy.load("en_core_web_trf")

True


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [ ]:
# View metadata of the model
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(nlp.meta)

{   'author': 'Explosion',
    'components': [   'transformer',
                      'tagger',
                      'parser',
                      'attribute_ruler',
                      'lemmatizer',
                      'ner'],
    'description': 'English transformer pipeline '
                   "(Transformer(name='roberta-base', "
                   "piece_encoder='byte-bpe', stride=104, type='roberta', "
                   'width=768, window=144, vocab_size=50265)). Components: '
                   'transformer, tagger, parser, ner, attribute_ruler, '
                   'lemmatizer.',
    'disabled': [],
    'email': 'contact@explosion.ai',
    'labels': {   'attribute_ruler': [],
                  'lemmatizer': [],
                  'ner': [   'CARDINAL',
                             'DATE',
                             'EVENT',
                             'FAC',
                             'GPE',
                             'LANGUAGE',
                             'LAW',

In [ ]:
# Print NER results for Descriptions
for i in range(9):
    print("Article",i)
    print(data.loc[i,"description"])
    print("Text Start End Label")
    doc = nlp(data.loc[i,"description"])
    for token in doc.ents:
        print(token.text, token.start_char,
              token.end_char, token.label_)
    print("\n")

Article 0
Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.
Text Start End Label


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Reuters 0 7 ORG


Article 1
Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Text Start End Label
Reuters 0 7 ORG
Carlyle Group 34 47 ORG


Article 2
Reuters - Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.
Text Start End Label
Reuters 0 7 ORG
next week 134 143 DATE
summer 168 174 DATE


Article 3
Reuters - Authorities have halted oil export flows from the main pipeline in southern Iraq after intelligence showed a rebel militia could strike infrastructure, an oil official said on Saturday.
Text Start End Label
Reuters 0 7 ORG
Iraq 86 90 GPE
Saturday 186 194 DATE


Article 4
AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months be

In [ ]:
doc.ents

(Forbes.com, PH.D., Sociology, Danny Bazil Riley, 70,000, 32, Riley)

## Train the Custom NER Model Using spaCy
Download config file at https://spacy.io/usage/training

In [43]:
# import os
# cwd = os.getcwd()
cwd = "/content/drive/MyDrive/XLLM_study/"

In [31]:
!python -m spacy init fill-config '/content/drive/MyDrive/XLLM_study/data/ag_dataset/base_config.cfg' '/content/drive/MyDrive/XLLM_study/data/ag_dataset/config.cfg'

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/XLLM_study/data/ag_dataset/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [46]:
# Auto-fill base template
ner_path = 'data/ag_dataset/ner/'

# The downloaded file from spaCy
config_file_path_input = cwd + ner_path + "config.cfg"

# The output file we will use for training
config_file_path_output = cwd + ner_path + "config_.cfg"

In [47]:
!python -m spacy init fill-config "$config_file_path_input" "$config_file_path_output"

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
/content/drive/MyDrive/XLLM_study/data/ag_dataset/ner/config_.cfg
You can now add your data and train your pipeline:
python -m spacy train config_.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [37]:
# Train spaCy model on NER annotations
output_path = cwd + "/models/ag_dataset/ner/ner-gpu-blank"
train_path = cwd + "/data/ag_dataset/ner/annotations/binary/train"
dev_path = cwd + "/data/ag_dataset/ner/annotations/binary/eval"

In [39]:
!python -m spacy train "$config_file_path_output" \
--output "$output_path" --paths.train "$train_path" \
--paths.dev "$dev_path" --training.max_epochs 30 --gpu-id 0 --verbose

[2025-03-12 07:03:02,222] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev', 'training.max_epochs']
✔ Created output directory: output_path
ℹ Saving to output directory: output_path
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-03-12 07:03:05,199] [INFO] Set up nlp object from config
[2025-03-12 07:03:05,214] [DEBUG] Loading corpus from path: dev_path
[2025-03-12 07:03:05,216] [DEBUG] Loading corpus from path: train_path
[2025-03-12 07:03:05,217] [INFO] Pipeline: ['transformer', 'ner']
[2025-03-12 07:03:05,219] [INFO] Created vocabulary
[2025-03-12 07:03:05,220] [INFO] Finished initializing nlp object
2025-03-12 07:03:07.116494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741762987.140134   32143 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register f